<a href="https://colab.research.google.com/github/harshitadd/AI4BharatTranslation/blob/main/urlPDF2TXT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install tesseract-ocr
!apt-get install poppler-utils
!pip install pytesseract
!pip install pdf2image


Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [2]:
import requests
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

In [3]:
# Data Source 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
with open('/content/gdrive/MyDrive/source_links.txt','r') as file:
  links = file.read()

In [5]:
links = links.split()

In [6]:
source_en_links = []
source_ta_links = []
for link in links:
  if '_e_' in link:
    source_en_links.append(link)
    temp = link 
    temp = temp.replace('_e_','_t_')
    source_ta_links.append(temp)


In [7]:
print(len(source_en_links))
print(len(source_ta_links))


431
431


In [8]:
i = 0
batched_links = []
# Making Mini Batches for download+preprocess
def minibatch(source_links):
  for j in range(0, len(source_links),20):
    batched_links.append(source_links[j:j+20])
  return batched_links 


In [9]:
def download_pdf_to_image(prim_lid, sec_lid, source_links, batch_index):
  source_counter = 1
  failed = []
  for source in source_links:
    url = source
    try: 
      response = requests.get(url)
      fn = '/content/gdrive/MyDrive/' + prim_lid + '_' + str(batch_index) + '_' + str(source_counter) + '.pdf' 
      with open(fn, 'wb') as f:
          f.write(response.content)
          pages = convert_from_path(fn, 200) 
          image_counter = 1
          skipped = 0
          for page in pages: 
            filename = "page_"+str(image_counter)+".jpg"
            page.save(filename, 'JPEG') 
            image_counter = image_counter + 1
          filelimit = image_counter-1
          print(str(source_counter) + ' PDF Succesfully Saved as Images')
  
          outfile = '/content/gdrive/MyDrive/' + prim_lid + '_' + str(batch_index) + '_' + str(source_counter) + '.txt' #Rewriting the original document with the processed version 
          f = open(outfile, "a") 
          for i in range(1, filelimit + 1):
            filename = "page_"+str(i)+".jpg" 
            try: 
              text = str(((pytesseract.image_to_string(Image.open(filename),lang=prim_lid+'+'+sec_lid))))  #  ORDER OF Languages Matter
            except:
              skipped+=1
              text =''
            f.write(text) 
          f.close() 
          source_counter+=1 
    except: 
      failed.append(url)
      print(url + ' PDF Failed to Process')
    print(str(source_counter) + ' PDF Succesfully Processed')
  
  print('Batch Complete!')
  print('Total Documents in :' + prim_lid + ' were ' + str(len(source_en_links)))
  print('URL fetch failed for :' + str(len(failed)))
  

In [ ]:
# main.py 

# CHANGE THIS
source_links = source_ta_links 
prim_lid = 'tam' # Primary lang
sec_lid = 'eng' # Depending upon which language is codeswitched.
batch_index = 0 #Change for different batches 
# # # 

batched_links = minibatch(source_links)
download_pdf_to_image(prim_lid, sec_lid, batched_links[batch_index], batch_index) # Downloading PDFs and storing them on Gdrive 

